##### Understand, in 3D context, torch-interpol and torch-functional affine matrix, grid and warping

#### Import

![image_info](./dicomCT.png)

In [1]:
#Imports
import os, sys, json, pathlib, shutil, glob
import pandas as pd
import csv
import SimpleITK as sitk
import nibabel as nib
import random
import math
import numpy as np
from PIL import Image, ImageFont, ImageDraw

import scipy
from scipy.io import loadmat
from scipy import  signal

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import ipywidgets as widgets
from ipywidgets import interactive,interact, interact_manual, HBox, Layout,VBox
from IPython.display import display, clear_output

import interpol
from interpol.api import affine_grid

from scipy import ndimage

from functools import partial

import torch
import torch.nn.functional as F
from torch.nn import MSELoss
from viu.io import volume
from viu.io.volume import read_volume
from viu.torch.deformation.fields import DVF, set_identity_mapping_cache
from viu.torch.io.deformation import *
from viu.util.body_mask import seg_body

# import ipywidgets as ipyw
import ipywidgets as widgets
from ipywidgets import interactive,interact, interact_manual, HBox, Layout,VBox
from IPython.display import display, clear_output

In [2]:
from exampleUtils import *

In [3]:
runTestCode=False
runTestCode_sec2=True
runTestCode_sec3=True

##### Read test volume

In [4]:
# experimentFolder = pathlib.Path('/mnt/data/supratik/diaphragm_detection/data/')
experimentFolder = pathlib.Path('/home/wd974888/Downloads/workingFolder/DeformationExperiment/')
patientMRN='Patient11AB_bin19'
dcmFolder = experimentFolder / patientMRN
vol, res, pos = read_volume(str(dcmFolder))
print(f'dcmFolder {dcmFolder}')
print(f'vol type {(type(vol))} shape {vol.shape} min {np.min(vol)} max {np.max(vol)} ')
print(f'res type {type(res)} value {res} ')
print(f'pos type {type(pos)} value {pos}')
toy_depth, toy_height, toy_width = vol.shape[0], vol.shape[1], vol.shape[2]
#Make the dimensions unequal
org_width = toy_width
toy_width = 8*toy_width//10
print(f'toy_depth {toy_depth}, toy_height {toy_height}, toy_width {toy_width}, org_width {org_width}')
vol = vol[:,:,:toy_width]
print(f'vol type {(type(vol))} shape {vol.shape}')
#With truncation in X direction, the pos matching the center of the volume in patient co-ordinate also shifts
old_pos_X = pos[0]
res_X = res[0]
new_pos_X = old_pos_X - 0.5*(org_width - toy_width)*res_X
pos[0] = new_pos_X
print(f'pos type {type(pos)} value {pos}')


dcmFolder /home/wd974888/Downloads/workingFolder/DeformationExperiment/Patient11AB_bin19
vol type <class 'numpy.ndarray'> shape (452, 512, 512) min -1890.0 max 36208.0 
res type <class 'numpy.ndarray'> value [0.9765625 0.9765625 1.       ] 
pos type <class 'numpy.ndarray'> value [ 1.71875000e-03 -2.09498281e+02 -9.00000000e-01]
toy_depth 452, toy_height 512, toy_width 409, org_width 512
vol type <class 'numpy.ndarray'> shape (452, 512, 409)
pos type <class 'numpy.ndarray'> value [ -50.29125    -209.49828125   -0.9       ]


In [5]:
v1_volumeComparisonViewer3D(
    listVolumes=[vol],listLabels=['original'],
    maxZ0=vol.shape[0], maxZ1=vol.shape[1], maxZ2=vol.shape[2],
    figsize=(12,8), cmap='gray',
    displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [6]:
align_corners=True
device=torch.device('cuda:0')
batchSize=1

In [8]:
#volume tensor with batch and channel
vol_tensor = torch.from_numpy(vol).to(device).unsqueeze(0).unsqueeze(0)
printTensor("vol_tensor", vol_tensor)


vol_tensor shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32


identity grid in 2D and in 3D in matrix (ij) and cartesian co-ordinate system

identity grid in 2D

In [9]:
if runTestCode:
    tmp_h, tmp_w = 5, 6
    print(f'tmp_h {tmp_h}, tmp_w {tmp_w}')
    #tmp_h 5, tmp_w 6
    tmp_IdGrid_matrix_2d = torch.stack(torch.meshgrid(*[torch.arange(s).float() for s in [tmp_h, tmp_w] ],indexing='ij'), -1).numpy().astype('float32')
    print(f'Pass shape [tmp_h {tmp_h}, tmp_w {tmp_w}]: tmp_IdGrid_matrix_2d type {(type(tmp_IdGrid_matrix_2d))} shape {tmp_IdGrid_matrix_2d.shape} dtype {tmp_IdGrid_matrix_2d.dtype}')

    tmp_IdGrid_cartesian_2d = torch.stack(torch.meshgrid(*[torch.arange(s).float() for s in [tmp_h, tmp_w] ],indexing='xy'), -1).numpy().astype('float32')
    print(f'Pass shape [tmp_h {tmp_h}, tmp_w {tmp_w}]: tmp_IdGrid_cartesian_2d type {(type(tmp_IdGrid_cartesian_2d))} shape {tmp_IdGrid_cartesian_2d.shape} dtype {tmp_IdGrid_cartesian_2d.dtype}')


identity grid in 3D

In [11]:
if runTestCode:
    tmp_d, tmp_h, tmp_w = 4, 5, 6
    print(f'tmp_d {tmp_d}, tmp_h {tmp_h}, tmp_w {tmp_w}')
    #tmp_d 4, tmp_h 5, tmp_w 6
    tmp_IdGrid_matrix = torch.stack(torch.meshgrid(*[torch.arange(s).float() for s in [tmp_d, tmp_h, tmp_w] ],indexing='ij'), -1).numpy().astype('float32')
    print(f'Pass shape [tmp_d {tmp_d}, tmp_h {tmp_h}, tmp_w {tmp_w}]: tmp_IdGrid_matrix type {(type(tmp_IdGrid_matrix))} shape {tmp_IdGrid_matrix.shape} dtype {tmp_IdGrid_matrix.dtype}')
    # Pass shape [tmp_d 4, tmp_h 5, tmp_w 6]: tmp_IdGrid_matrix type <class 'numpy.ndarray'> shape (4, 5, 6, 3) dtype float32
    #tmp_IdGrid_matrix[k,:,:,0]: Channel 0 all elements filled with z-coordinate (namely k) of the slice: for all slice k=0,1,...,toy_d-1 
    #tmp_IdGrid_matrix[k,:,:,1]: Channel 1 columns filled with  (i) y-coordinates: all columns  are transpose([0,1,... toy_h-1]), for all slice k=0,1,...,toy_d-1 
    #tmp_IdGrid_matrix[k,:,:,2]: Channel 2 rows filled with  (j) x-coordinates:  all rows  are [0,1,... toy_w-1],, for all slice  k=0,1,...,toy_d-1 
    tmp_IdGrid_cartesian = torch.stack(torch.meshgrid(*[torch.arange(s).float() for s in [tmp_d, tmp_h, tmp_w] ],indexing='xy'), -1).numpy().astype('float32')
    print(f'Pass shape [tmp_d {tmp_d}, tmp_h {tmp_h}, tmp_w {tmp_w}]: tmp_IdGrid_cartesian type {(type(tmp_IdGrid_cartesian))} shape {tmp_IdGrid_cartesian.shape} dtype {tmp_IdGrid_cartesian.dtype}')
    # Pass shape [tmp_d 4, tmp_h 5, tmp_w 6]: tmp_IdGrid_cartesian type <class 'numpy.ndarray'> shape (5, 4, 6, 3) dtype float32
    print('If we assumed, 1st dim is depth then in generated cartesian mesgrid, 2nd dimension corresponds to depth.')
    #If we assumed, 1st dim is depth then in generated cartesian mesgrid, 2nd dimension corresponds to depth.
    #tmp_IdGrid_cartesian[:,k,:,0]: Channel 0 all elements filled with z-coordinate (namely k) of the slice: for all slice k=0,1,...,toy_d-1 
    #tmp_IdGrid_cartesian[:,k,:,1]: Channel 1 columns filled with  (i) y-coordinates: all columns  are transpose([0,1,... toy_h-1]), for all slice k=0,1,...,toy_d-1 
    #tmp_IdGrid_cartesian[:,k,:,2]: Channel 2 rows filled with  (j) x-coordinates:  all rows  are [0,1,... toy_w-1],, for all slice  k=0,1,...,toy_d-1 

##### Test steps

For rotation around Z, Y, X axis passing through center of volume

    - Find affine matrix  for torch-interpol grid

    - Create torch-interpol push  grid and apply torch-interpol grid_push

    - Create torch-interpol pull  grid and apply torch-interpol grid_pull

    - Convert torch-interpol pull grid to torch functional pull grid and  apply torch functional grid_warp(pull)

    - Convert  torch functional pull grid  back to torch-interpol pull grid  and apply torch-interpol grid_pull

Create affine matrix

In [10]:
# Rotation around Dicom Z axis : As viewed into Axial plane    from origin toward +Z axis (towards H(Head)), with A (Anterior) on top of the view, pass clockwise rotation is positive
# Rotation around Dicom Y axis : As viewed into Coronal plane  from origin toward +Y axis (towards P(Posterior)), with H (Head) on top of the view, pass clockwise rotation is positive
# Rotation around Dicom X axis : As viewed  (lying down horizontally) into Sagittal plane from +X toward origin (towards R(Right)) with H (Head) on top of the view, pass clockwise rotation is positive
theta_deg = 15.00 #Rotate image  counter clockwise by 10 degree
viewString='coronal'
pushAffine_np = getPushRotationMatrix(
    theta_deg=theta_deg, 
    viewString=viewString,
    center_slice_z=toy_depth/2.0,
    center_row_y=toy_height/2.0, 
    center_col_x=toy_width/2.0)
print(f'pushAffine_np {pushAffine_np}')
pullAffine_np = np.linalg.inv(pushAffine_np)
print(f'pullAffine_np {pullAffine_np}')

pushAffine_np [[  0.9659258    0.          -0.25881904  60.629257  ]
 [  0.           1.           0.           0.        ]
 [  0.25881904   0.           0.9659258  -51.524933  ]
 [  0.           0.           0.           1.        ]]
pullAffine_np [[  0.9659258    0.           0.25881904 -45.227734  ]
 [  0.           1.           0.           0.        ]
 [ -0.25881904   0.           0.9659258   65.46127   ]
 [  0.           0.           0.           1.        ]]


Create torch interpol push grid and apply  push warp

In [11]:
if runTestCode:
    pushAffine_unBatchedTensor = torch.from_numpy(pushAffine_np).to(device)
    printTensor("pushAffine_unBatchedTensor", pushAffine_unBatchedTensor)
    aRotGrid_push_unbatched = affine_grid(pushAffine_unBatchedTensor, [toy_depth, toy_height, toy_width])
    printTensor("aRotGrid_push_unbatched", aRotGrid_push_unbatched)
    #Expand to batch
    aRotGrid_push_batched = aRotGrid_push_unbatched.expand(batchSize, *aRotGrid_push_unbatched.shape)
    printTensor("aRotGrid_push_batched", aRotGrid_push_batched)

In [12]:
if runTestCode:
   aVol_warp_interpol_push_tensor = interpol.grid_push(vol_tensor, aRotGrid_push_batched, 
      interpolation='cubic', bound='zeros',prefilter=True)
   printTensor("aVol_warp_interpol_push_tensor", aVol_warp_interpol_push_tensor)
   aVol_warp_interpol_push_np = aVol_warp_interpol_push_tensor.clone().squeeze(0).squeeze().cpu().numpy()
   print(f"aVol_warp_interpol_push_np shape {aVol_warp_interpol_push_np.shape} dtype {aVol_warp_interpol_push_np.dtype}")
   v1_volumeComparisonViewer3D(
      listVolumes=[aVol_warp_interpol_push_np],listLabels=[f'Interpol_PushWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
      maxZ0=aVol_warp_interpol_push_np.shape[0], maxZ1=aVol_warp_interpol_push_np.shape[1], maxZ2=aVol_warp_interpol_push_np.shape[2],
      figsize=(12,8), cmap='gray',
      displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

Create torch interpol pull grid and apply pull warp

In [13]:
if runTestCode:
    pullAffine_unBatchedTensor = torch.from_numpy(pullAffine_np).to(device)
    printTensor("pullAffine_unBatchedTensor", pullAffine_unBatchedTensor)
    aRotGrid_pull_unbatched = affine_grid(pullAffine_unBatchedTensor, [toy_depth, toy_height, toy_width])
    printTensor("aRotGrid_pull_unbatched", aRotGrid_pull_unbatched)
    #Expand to batch
    aRotGrid_pull_batched = aRotGrid_pull_unbatched.expand(batchSize, *aRotGrid_pull_unbatched.shape)
    printTensor("aRotGrid_pull_batched", aRotGrid_pull_batched)

In [14]:
if runTestCode:
   aVol_warp_interpol_pull_tensor = interpol.grid_pull(vol_tensor, aRotGrid_pull_batched, 
      interpolation='cubic', bound='zeros',prefilter=True)
   printTensor("aVol_warp_interpol_pull_tensor", aVol_warp_interpol_pull_tensor)
   aVol_warp_interpol_pull_np = aVol_warp_interpol_pull_tensor.clone().squeeze().squeeze().cpu().numpy()
   print(f"aVol_warp_interpol_pull_np shape {aVol_warp_interpol_pull_np.shape} dtype {aVol_warp_interpol_pull_np.dtype}")
   v1_volumeComparisonViewer3D(
      listVolumes=[aVol_warp_interpol_pull_np],listLabels=[f'Interpol_pullWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
      maxZ0=aVol_warp_interpol_pull_np.shape[0], maxZ1=aVol_warp_interpol_pull_np.shape[1], maxZ2=aVol_warp_interpol_pull_np.shape[2],
      figsize=(12,8), cmap='gray',
      displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

#### Direct conversion from torch interpol pull grid to Torch functional pull grid by normalization

Create torch functional pull grid  from torch interpol pull grid and apply  PyTorch pull warp

In [15]:
if runTestCode:
    aTorchFunctionalPullGrid_batched = convertGrid_interpol2functional(aRotGrid_pull_batched,toy_depth, toy_height, toy_width)
    printTensor("aTorchFunctionalPullGrid_batched", aTorchFunctionalPullGrid_batched)
    aVol_warp_functional_pull_tensor = F.grid_sample(
        input=vol_tensor,
        grid=aTorchFunctionalPullGrid_batched,
        mode='bilinear',
        padding_mode='zeros',
        align_corners=align_corners)
    printTensor("aVol_warp_functional_pull_tensor", aVol_warp_functional_pull_tensor)
    aVol_warp_functional_pull_np = aVol_warp_functional_pull_tensor.clone().squeeze().squeeze().cpu().numpy()
    print(f"aVol_warp_functional_pull_np shape {aVol_warp_functional_pull_np.shape} dtype {aVol_warp_functional_pull_np.dtype}")
    v1_volumeComparisonViewer3D(
        listVolumes=[aVol_warp_functional_pull_np],listLabels=[f'Fuctional_PullWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
        maxZ0=aVol_warp_functional_pull_np.shape[0], maxZ1=aVol_warp_functional_pull_np.shape[1], maxZ2=aVol_warp_functional_pull_np.shape[2],
        figsize=(12,8), cmap='gray',
        displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

#### Direct conversion from torch functional pull grid to torch interpol pull grid to  by de-normalization

Create torch interpol pull grid from  torch functional pull grid  and apply  torch interpol pull warp

In [16]:
if runTestCode:
   aRotGrid_pull_batched_from_converted = convertGrid_functional2interpol(aTorchFunctionalPullGrid_batched, toy_depth, toy_height, toy_width)
   aVol_warp_interpol_pull_from_converted = interpol.grid_pull(vol_tensor, aRotGrid_pull_batched_from_converted, 
      interpolation='cubic', bound='zeros',prefilter=True)
   printTensor("aVol_warp_interpol_pull_from_converted", aVol_warp_interpol_pull_from_converted)
   aVol_warp_interpol_pull_from_converted_np = aVol_warp_interpol_pull_from_converted.clone().squeeze().squeeze().cpu().numpy()
   print(f"aVol_warp_interpol_pull_from_converted_np shape {aVol_warp_interpol_pull_from_converted_np.shape} dtype {aVol_warp_interpol_pull_from_converted_np.dtype}")
   v1_volumeComparisonViewer3D(
      listVolumes=[aVol_warp_interpol_pull_from_converted_np],listLabels=[f'F2InterpolConverted_pullWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
      maxZ0=aVol_warp_interpol_pull_from_converted_np.shape[0], maxZ1=aVol_warp_interpol_pull_from_converted_np.shape[1], maxZ2=aVol_warp_interpol_pull_from_converted_np.shape[2],
      figsize=(12,8), cmap='gray',
      displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

##### Test steps

-   Create torch-functional affine mat from un-normalized  torch-interpol affine matrix. Create both push and pull type affine mat

-   Create Torch functional pull grid from pull affine Mat, apply warp and display. Note torch functional has only pull-warp.

-   Create VIU DVF push grid and pull grid from push and pull affine Mat.

-   Apply VIU DVF pull warp  without prefilter using pull grid and display. Internally torch  functional pull-warp will be used.

-   Apply VIU DVF  pull warp WITH prefilter using pull grid and display. Internally torch-interpol pull-warp will be used as prefilter=True.

-   Apply VIU DVF  push warp WITH prefilter using push grid and display. Internally torch-interpol pull-warp will be used as (push-warp= | prefilter=True).



Create torch-functional affine mat from un-normalized  torch-interpol affine matrix. Create both push and pull type affine mat

In [17]:
pyTorchPushAffineMat3x4 = getPyTorchAffineMatTensor(pushAffine_np,  toy_depth, toy_height, toy_width, toy_depth, toy_height, toy_width, device)
printTensor('pyTorchPushAffineMat3x4', pyTorchPushAffineMat3x4)
print(f'pyTorchPushAffineMat3x4 {pyTorchPushAffineMat3x4}')
pyTorchPullAffineMat3x4 = getPyTorchAffineMatTensor(pullAffine_np,  toy_depth, toy_height, toy_width, toy_depth, toy_height, toy_width, device)
printTensor('pyTorchPullAffineMat3x4', pyTorchPullAffineMat3x4)
print(f'pyTorchPullAffineMat3x4 {pyTorchPullAffineMat3x4}')

pyTorchPushAffineMat3x4 shape: torch.Size([3, 4]) device: cuda:0 dtype: torch.float32
pyTorchPushAffineMat3x4 tensor([[ 9.6593e-01,  0.0000e+00,  2.8603e-01,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.3420e-01,  0.0000e+00,  9.6593e-01, -1.1921e-07]], device='cuda:0')
pyTorchPullAffineMat3x4 shape: torch.Size([3, 4]) device: cuda:0 dtype: torch.float32
pyTorchPullAffineMat3x4 tensor([[ 9.6593e-01,  0.0000e+00, -2.8603e-01, -1.1921e-07],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 2.3420e-01,  0.0000e+00,  9.6593e-01, -5.9605e-08]], device='cuda:0')


Create VIU DVF push grid and pull grid from push and pull affine Mat.

In [18]:
# from viu.torch.deformation.fields import DVF
printTensor('pyTorchPushAffineMat3x4', pyTorchPushAffineMat3x4)
printTensor('pyTorchPullAffineMat3x4', pyTorchPullAffineMat3x4)
nb_dim = pyTorchPullAffineMat3x4.shape[-1]-1
viuDVFGridSizeParam=(batchSize, 1, toy_depth, toy_height, toy_width, nb_dim)
print(f'viuDVFGridSizeParam shape: (batch, channel, D, H, W, nbDim=3): {viuDVFGridSizeParam}')

#While calling DVF.affine, in the affine matrix, the last row (0,0,1) for 2D or (0,0,0,1) for is removed. However no batch is added.
#Also  while passing the size, the  DVF  dimension  (2 or 3) is being passed  as the last elemet of the size  tuple.
#Further  line #393 of field.py  is to be modified.
#Another important difference  is that  the DVF returned by DVF.affine  includes a subtraction of the identity map
viu_grid_push = DVF.affine(pyTorchPushAffineMat3x4, size=viuDVFGridSizeParam)
printTensor('viu_grid_push', viu_grid_push)
viu_grid_pull = DVF.affine(pyTorchPullAffineMat3x4, size=viuDVFGridSizeParam)
printTensor('viu_grid_pull', viu_grid_pull)

pyTorchPushAffineMat3x4 shape: torch.Size([3, 4]) device: cuda:0 dtype: torch.float32
pyTorchPullAffineMat3x4 shape: torch.Size([3, 4]) device: cuda:0 dtype: torch.float32
viuDVFGridSizeParam shape: (batch, channel, D, H, W, nbDim=3): (1, 1, 452, 512, 409, 3)
viu_grid_push shape: torch.Size([1, 452, 512, 409, 3]) device: cuda:0 dtype: torch.float32
viu_grid_pull shape: torch.Size([1, 452, 512, 409, 3]) device: cuda:0 dtype: torch.float32


Create Torch functional pull grid from pull affine Mat, apply warp and display. Note torch functional has only pull-warp.

In [19]:
if runTestCode_sec2:
    #Torch functional affine grid generation requires affine matrix with batch dimension
    pyTorchPullAffineMat3x4_batchAdded = pyTorchPullAffineMat3x4.expand(batchSize, *pyTorchPullAffineMat3x4.shape)
    printTensor('pyTorchPullAffineMat3x4_batchAdded', pyTorchPullAffineMat3x4_batchAdded)
    # print(f'pyTorchThetaForAffineGrid_lastRowRemoved_batchAdded type {type(pyTorchThetaForAffineGrid_lastRowRemoved_batchAdded)} shape {pyTorchThetaForAffineGrid_lastRowRemoved_batchAdded.shape}')
    # print(f'pyTorchThetaForAffineGrid_lastRowRemoved_batchAdded  {pyTorchThetaForAffineGrid_lastRowRemoved_batchAdded}')
    grid_size_F = torch.Size([batchSize, 1, toy_depth, toy_height, toy_width]) #batch, channel, depth, height, width
    print(f'grid_size_F {grid_size_F}')
    grid_F = F.affine_grid(
        theta=pyTorchPullAffineMat3x4_batchAdded,
        size=grid_size_F,
        align_corners=align_corners)
    print(f'grid_F type {type(grid_F)} shape {grid_F.shape} dtype {grid_F.dtype}')
    aVol_warp_functional_pullUsingMat_tensor = F.grid_sample(
        input=vol_tensor,
        grid=grid_F,
        mode='bilinear',
        padding_mode='zeros',
        align_corners=align_corners)
    printTensor("aVol_warp_functional_pullUsingMat_tensor", aVol_warp_functional_pullUsingMat_tensor)
    aVol_warp_functional_pullUsingMat_np = aVol_warp_functional_pullUsingMat_tensor.clone().squeeze().squeeze().cpu().numpy()
    print(f"aVol_warp_functional_pullUsingMat_np shape {aVol_warp_functional_pullUsingMat_np.shape} dtype {aVol_warp_functional_pullUsingMat_np.dtype}")
    v1_volumeComparisonViewer3D(
        listVolumes=[aVol_warp_functional_pullUsingMat_np],listLabels=[f'Fuctional_PullWarpUsingMat_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
        maxZ0=aVol_warp_functional_pullUsingMat_np.shape[0], maxZ1=aVol_warp_functional_pullUsingMat_np.shape[1], maxZ2=aVol_warp_functional_pullUsingMat_np.shape[2],
        figsize=(12,8), cmap='gray',
        displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

pyTorchPullAffineMat3x4_batchAdded shape: torch.Size([1, 3, 4]) device: cuda:0 dtype: torch.float32
grid_size_F torch.Size([1, 1, 452, 512, 409])
grid_F type <class 'torch.Tensor'> shape torch.Size([1, 452, 512, 409, 3]) dtype torch.float32
aVol_warp_functional_pullUsingMat_tensor shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32
aVol_warp_functional_pullUsingMat_np shape (452, 512, 409) dtype float32


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Apply VIU DVF pull warp  without prefilter using pull grid and display. Internally torch  functional pull-warp will be used.

In [20]:
if runTestCode_sec2:
    printTensor('viu_grid_pull', viu_grid_pull)
    printTensor('vol_tensor', vol_tensor)
    #sample(self, input, mode="bilinear", padding_mode="zeros", warpingModeString="pull",prefilter=False)
    aVol_viu_grid_pull_warped = viu_grid_pull.sample(
        input=vol_tensor,
        mode='bilinear',
        padding_mode='zeros',
        warpingModeString="pull",
        prefilter=False)
    printTensor('aVol_viu_grid_pull_warped', aVol_viu_grid_pull_warped)
    aVol_viu_grid_pull_warped_np = aVol_viu_grid_pull_warped.squeeze().squeeze().clone().cpu().numpy()
    v1_volumeComparisonViewer3D(
        listVolumes=[aVol_viu_grid_pull_warped_np],listLabels=[f'VIUDVF_PullWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
        maxZ0=aVol_viu_grid_pull_warped_np.shape[0], maxZ1=aVol_viu_grid_pull_warped_np.shape[1], maxZ2=aVol_viu_grid_pull_warped_np.shape[2],
        figsize=(12,8), cmap='gray',
        displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

viu_grid_pull shape: torch.Size([1, 452, 512, 409, 3]) device: cuda:0 dtype: torch.float32
vol_tensor shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32
aVol_viu_grid_pull_warped shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Apply VIU DVF  pull warp WITH prefilter using pull grid and display. Internally torch-interpol pull-warp will be used as prefilter=True.

In [21]:
if runTestCode_sec2:
    printTensor('viu_grid_pull', viu_grid_pull)
    printTensor('vol_tensor', vol_tensor)
    #sample(self, input, mode="bilinear", padding_mode="zeros", warpingModeString="pull",prefilter=False)
    aVol_viu_grid_pull_prefilter_warped = viu_grid_pull.sample(
        input=vol_tensor,
        mode='cubic',
        padding_mode='zeros',
        warpingModeString="pull",
        prefilter=True)
    printTensor('aVol_viu_grid_pull_prefilter_warped', aVol_viu_grid_pull_prefilter_warped)
    aVol_viu_grid_pull_prefilter_warped_np = aVol_viu_grid_pull_prefilter_warped.squeeze().squeeze().clone().cpu().numpy()
    v1_volumeComparisonViewer3D(
        listVolumes=[aVol_viu_grid_pull_prefilter_warped_np],listLabels=[f'VIUDVF_pull_prefilterWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
        maxZ0=aVol_viu_grid_pull_prefilter_warped_np.shape[0], maxZ1=aVol_viu_grid_pull_prefilter_warped_np.shape[1], maxZ2=aVol_viu_grid_pull_prefilter_warped_np.shape[2],
        figsize=(12,8), cmap='gray',
        displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

viu_grid_pull shape: torch.Size([1, 452, 512, 409, 3]) device: cuda:0 dtype: torch.float32
vol_tensor shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32


/opt/conda/envs/clone2MMIPython/lib/python3.9/site-packages/interpol/autograd.py:287: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/third_party/nvfuser/csrc/parser.cpp:3820.)
  output = spline_coeff_nd(input, *opt)


aVol_viu_grid_pull_prefilter_warped shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Apply VIU DVF  push warp WITH prefilter using push grid and display. Internally torch-interpol pull-warp will be used as (push-warp= | prefilter=True).

In [22]:
if runTestCode_sec2:
    printTensor('viu_grid_push', viu_grid_push)
    printTensor('vol_tensor', vol_tensor)
    #sample(self, input, mode="bilinear", padding_mode="zeros", warpingModeString="pull",prefilter=False)
    aVol_viu_grid_push_prefilter_warped = viu_grid_push.sample(
        input=vol_tensor,
        mode='cubic',
        padding_mode='zeros',
        warpingModeString="push",
        prefilter=True)
    printTensor('aVol_viu_grid_push_prefilter_warped', aVol_viu_grid_push_prefilter_warped)
    aVol_viu_grid_push_prefilter_warped_np = aVol_viu_grid_push_prefilter_warped.squeeze().squeeze().clone().cpu().numpy()
    v1_volumeComparisonViewer3D(
        listVolumes=[aVol_viu_grid_push_prefilter_warped_np],listLabels=[f'VIUDVF_push_prefilterWarp_Rot_by_{theta_deg:0.1f}_deg_around_{viewString}'],
        maxZ0=aVol_viu_grid_push_prefilter_warped_np.shape[0], maxZ1=aVol_viu_grid_push_prefilter_warped_np.shape[1], maxZ2=aVol_viu_grid_push_prefilter_warped_np.shape[2],
        figsize=(12,8), cmap='gray',
        displayColorbar=False, useExternalWindowCenter=True, wMin=-500, wMax=500)

viu_grid_push shape: torch.Size([1, 452, 512, 409, 3]) device: cuda:0 dtype: torch.float32
vol_tensor shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32


/opt/conda/envs/clone2MMIPython/lib/python3.9/site-packages/interpol/autograd.py:287: UserWarning: operator() profile_node %258 : int[] = prim::profile_ivalue(%permutation.21)
 does not have profile information (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  output = spline_coeff_nd(input, *opt)


aVol_viu_grid_push_prefilter_warped shape: torch.Size([1, 1, 452, 512, 409]) device: cuda:0 dtype: torch.float32


interactive(children=(Output(),), _dom_classes=('widget-interact',))

#### Timing consideration

Execute each of interpol grid pull and  functional grid pull N times and compare time taken

In [25]:
numRepeat = 10
import time

In [26]:
if runTestCode_sec3:
    start = time.time()
    for count in range(numRepeat):
        aVol_viu_grid_pull_warped = viu_grid_pull.sample(
            input=vol_tensor,
            mode='bilinear',
            padding_mode='zeros',
            warpingModeString="pull",
            prefilter=False)
    end = time.time()
    print(f'Volume shape {vol.shape}, numRepeat {numRepeat} : VIU DVF Pull + bilinear + no-PreFilter => (Internally F.grid_sample) =>: {(end - start):.2f} seconds')

Volume shape (452, 512, 409), numRepeat 10 : VIU DVF Pull + bilinear + no-PreFilter => (Internally F.grid_sample) =>: 0.01 seconds


In [27]:
if runTestCode_sec3:
    start = time.time()
    for count in range(numRepeat):
        Vol_viu_grid_pull_prefilter_warped = viu_grid_pull.sample(
            input=vol_tensor,
            mode='cubic',
            padding_mode='zeros',
            warpingModeString="pull",
            prefilter=True)
    end = time.time()
    print(f'Volume shape {vol.shape}, numRepeat {numRepeat} : VIU DVF Pull + cubic + PreFilter => (Internally preFilter + interpol.grid_pull) =>: {(end - start):.2f} seconds')

Volume shape (452, 512, 409), numRepeat 10 : VIU DVF Pull + cubic + PreFilter => (Internally preFilter + interpol.grid_pull) =>: 12.00 seconds


In [28]:
if runTestCode_sec3:
    start = time.time()
    for count in range(numRepeat):
        aVol_viu_grid_push_prefilter_warped = viu_grid_push.sample(
            input=vol_tensor,
            mode='cubic',
            padding_mode='zeros',
            warpingModeString="push",
            prefilter=True)
    end = time.time()
    print(f'Volume shape {vol.shape}, numRepeat {numRepeat} : VIU DVF Push + cubic + PreFilter => (Internally preFilter + interpol.grid_push) =>: {(end - start):.2f} seconds')

Volume shape (452, 512, 409), numRepeat 10 : VIU DVF Push + cubic + PreFilter => (Internally preFilter + interpol.grid_push) =>: 12.41 seconds
